In [2]:
import numpy as np

# Suppose arr is loaded from Boston.csv as in your notebook:
# arr = np.genfromtxt('data/Boston.csv', delimiter=',', names=True, dtype=None, encoding='utf-8')
# Y_Train = arr["MEDV"]
# fields = [name for name in arr.dtype.names if name != "MEDV"]
# X_Train = arr[fields]  # structured array with columns = features

# For illustration, let’s assume X_Train is a structured array of shape (m,)
# and wList is initialized to 1.0 for each feature, b = 0, learning rate a = 0.0001:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
arr = np.genfromtxt('data/Boston.csv', delimiter=',', names=True, dtype=None, encoding='utf-8')
Y_Train = arr["MEDV"]
fields = list(arr.dtype.names)
fields.remove('MEDV')
X_Train = arr[fields]

wList = [1.0] * len(fields)
b = 0.0
a = 0.0001
m = X_Train.shape[0]

def model(wList, fields, xValues, b):
    """Compute w·x + b for a single example xValues."""
    total = b
    for idx in range(len(wList)):
        total += wList[idx] * xValues[fields[idx]]
    return total

def derCostFunc(wList, X_Train, Y_Train, fields, b, a):
    m = X_Train.shape[0]
    for i in range(len(wList)):
        der = 0.0
        for j in range(m):
            xj = X_Train[j]                       # one example (structured row)
            pred_j = model(wList, fields, xj, b)  # scalar prediction
            error_j = pred_j - Y_Train[j]         # scalar error
            xi_j = xj[fields[i]]
            print(xi_j)# scalar feature x^{(j)}_i
            der += error_j * xi_j                 # accumulate scalar

        der /= m
        wList[i] -= a * der
    return wList

# Run one gradient step:
wList = derCostFunc(wList, X_Train, Y_Train, fields, b, a)
print("Updated weights:", wList)

0.00632
0.02731
0.02729
0.032369999999999996
0.06905
0.02985
0.08829
0.14455
0.21124
0.17004
0.22489
0.11747
0.09378
0.62976
0.6379600000000001
0.62739
1.05393
0.7842
0.80271
0.7258
1.25179
0.8520399999999999
1.2324700000000002
0.9884299999999999
0.75026
0.84054
0.67191
0.9557700000000001
0.77299
1.00245
1.13081
1.3547200000000001
1.38799
1.15172
1.6128200000000001
0.06417
0.09744
0.08014
0.17505
18.0
0.0
0.0
0.0
0.0
0.0
12.5
12.5
12.5
12.5
12.5
12.5
12.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.31
7.07
7.07
2.18
2.18
2.18
7.87
7.87
7.87
7.87
7.87
7.87
7.87
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
8.14
5.96
5.96
5.96
5.96
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.538
0.469
0.469
0.458
0.458
0.458
0.524
0.524
0.524
0.524
0.524
0.524
0.524
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0.538
0